# Inventário

Deve-se executar as seis primeiras células para carregar funções utilizadas para rodar o script

In [2]:
import json
import os
from openpyxl import load_workbook
from datetime import timedelta
import random
from copy import deepcopy
from django.contrib.gis.geos import LineString, Point
from django.contrib.admin.utils import flatten
import math
import datetime
import locale
from tqdm.notebook import tqdm
from PIL import Image
from io import BytesIO
from django.contrib.gis.geos import Point, LineString as DjangoLineString
import pytz
from simple_history.utils import bulk_create_with_history
from helpers.histories import bulk_update_with_history
from helpers.route_maker import dic_to_ordered_list, unequal_point_pairs

In [3]:
#encontra a latitude e longitude e retorna 

def km_to_coordinates(road, km):

    road_marks = dic_to_ordered_list(road.marks)
    selected_pair = None

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] > km and pair[1]["km"] < km:
            selected_pair = pair
            break
        elif pair[1]["km"] > km and pair[0]["km"] < km:
            selected_pair = pair
            break
        elif pair[0]["km"] == km:
            return Point(pair[0]["point"]["coordinates"]), road
        elif pair[1]["km"] == km:
            return Point(pair[1]["point"]["coordinates"]), road

    # Calculate total length
    start_km = min([selected_pair[0]["km"], selected_pair[1]["km"]])
    end_km = max([selected_pair[0]["km"], selected_pair[1]["km"]])
    segment_mark_length = end_km - start_km
    distance_from_min = km - start_km
    # Check if km is decreasing
    invert_km = selected_pair[1]["km"] <= selected_pair[0]["km"]

    # Cut segment
    start_key = min((selected_pair[0]["index"], selected_pair[1]["index"]))
    end_key = max((selected_pair[0]["index"], selected_pair[1]["index"]))

    points = []
    for key in range(start_key, end_key + 1):
        points.append(
            Point(road.path[key][0], road.path[key][1], road.path[key][2])
        )

    segment = LineString(points, srid=4326)
    # Apply km inversion correction
    if invert_km:
        segment_distance = (
            1 - (distance_from_min / segment_mark_length)
        ) * segment.length
    else:
        segment_distance = (
            distance_from_min / segment_mark_length
        ) * segment.length
    # Find point
    point = segment.interpolate(segment_distance)

    return Point(point.x, point.y), road

In [4]:
#verifica se o km está dentro da rodovia

def check_valid_road(road, km):

    road_marks = dic_to_ordered_list(road.marks)

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] >= km and pair[1]["km"] <= km:
            return True
        elif pair[1]["km"] >= km and pair[0]["km"] <= km:
            return True

    return False

In [5]:
def get_connected_reference(company, resource, key, value):
    value=value.strip()
    select_options = company.custom_options[resource]['fields'][key]['selectOptions']['options']
    return next(a['value'] for a in select_options if a['name'].lower() == value.lower())

In [6]:
def get_road_coordinates(road_name, km, direction, company):
    road_set = Road.objects.filter(
        name__contains=road_name, direction=int(direction), company=company
    )
#     print(road_set)

    # If roads in specified direction are not found, search road only
    # by name and order then by direction
    road_set_generic = Road.objects.filter(
        name__contains=road_name, company=company
    ).order_by("direction")

    # Check if KM range in road_set
#     print("Searching KM on direction...")
    valid = False
    for road in road_set:
        if check_valid_road(road, km):
#             print("Found KM on Road {}".format(road))
            valid = True
            break

    if not valid:
#         print("Searching KM without direction...")
        for road in road_set_generic:
            if check_valid_road(road, km):
#                 print("Found KM on Road {}".format(road))
                valid = True
                break

    if not valid:
        return Point(0, 0), None

    try:
        return km_to_coordinates(road, km)
    except Exception:
        return Point(0, 0), None

In [7]:
get_road_coordinates('ERS-122',0.37,'0',Company.objects.get(name='Caminhos da Serra Gaúcha'))

(<Point object at 0x7059d4ceac10>, <Road: 2059: ERS-122>)

In [8]:
def findIndex(flist, func):
    for i,v in enumerate(flist):
        if func(v): 
            return i
    return -1

In [9]:
def point_to_km(lng, lat, road):
    point = Point(lng, lat, srid=4326)
    if isinstance(road,str):
        roads = Road.objects.filter(company=company,name__icontains=road).annotate(distance=Distance(point, 'path')).order_by('distance')
        road = roads[0]
    else:
        road = roads[0]
    path = loads(road.path.geojson)
    point_geojson = loads(point.geojson)
    road_marks = list(road.marks.values())
    snap_point = nearest_point_on_line(path, point_geojson)
    

    road_markers = sorted([a for a in road_marks if 'index' in a or ('index' in a and a['index'] == 0)], key=lambda x: int(x['key']))
#     road_markers = sorted([a for a in road_marks if a['index'] or a['index'] == 0], key=lambda x: int(x['key']))
    rm_index = findIndex(road_markers, lambda x: x['index'] > snap_point['properties']['index'])
    
    if (rm_index == -1):
        rm_index = len(road_markers) - 1;
    
    km_marker_start = road_markers[rm_index - 1]
    km_marker_end = road_markers[rm_index]
    
    if km_marker_start['index'] == road_markers[-1]['index']: 
        snap = road_markers[-1]
        km = snap['km']
    elif km_marker_end['index'] == road_markers[0]['index']:
        snap = road_markers[0]
        km = snap['km']
    else:
        test = path['coordinates'][km_marker_start['index']:km_marker_end['index'] + 1]
        marker_path = LineString(test)
        snap = nearest_point_on_line(marker_path, point_geojson)        
        ls_length = length(marker_path, units="km")
        km = ((snap['properties']['location'] / ls_length)        
            * (km_marker_end['km'] - km_marker_start['km'])
            + km_marker_start['km'])
    
    return road, road.name, point, round(km, 3)

In [10]:
from collections import defaultdict
from django.db.models.signals import *


class DisableSignals(object):
    def __init__(self, disabled_signals=None):
        self.stashed_signals = defaultdict(list)
        self.disabled_signals = disabled_signals or [
            pre_init, post_init,
            pre_save, post_save,
            pre_delete, post_delete,
            pre_migrate, post_migrate,
        ]

    def __enter__(self):
        for signal in self.disabled_signals:
            self.disconnect(signal)

    def __exit__(self, exc_type, exc_val, exc_tb):
        for signal in list(self.stashed_signals):
            self.reconnect(signal)

    def disconnect(self, signal):
        self.stashed_signals[signal] = signal.receivers
        signal.receivers = []

    def reconnect(self, signal):
        signal.receivers = self.stashed_signals.get(signal, [])
        del self.stashed_signals[signal]

In [11]:
def get_sign(field, sign_field, occ, value):
#     if sign_field == 'Cor':
#         value=value.capitalize() if value != 'Amarela fluorescente (VII)' else value
#     elif sign_field == 'Tipo de Película':
#         value=value
    values = occ.form_fields['fields']
    item_translation = flatten([item['innerFields'] for item in values if item['displayName']==field])
#     print(item_translation)
    mid_translation = flatten([item['selectOptions']['options'] for item in item_translation if item['displayName'] == sign_field])
#     print(mid_translation)
    final_translation = {item['name']: item['value'] for item in mid_translation}
#     print(final_translation)
    return final_translation[value]

In [12]:
def get_value(field, occ, value):
    values = occ.form_fields['fields']
    item_translation = flatten([item['selectOptions']['options'] for item in values if item['displayName']==field])
#     print(item_translation)

    final_translation = {item['name']: item['value'] for item in item_translation}
#     print(final_translation)
    return final_translation[value]

 Verificando se está no ambiente de produção

In [13]:
!cat .env

cat: .env: No such file or directory


Escolhendo nome do arquivo Excel para carregar o Inventário

In [14]:
filename='Sinalizacao Vertical_CSG_Kartado parte 1'
road_name='ERS-122'

Escolhendo a aba da planilha do Excel

In [15]:
wb = load_workbook(filename + '.xlsx')
sheetname = wb.sheetnames[1]
ws = wb[sheetname]
ws

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


<Worksheet "Sinalização Vertical">

Testando planilha carregada do Excel

In [16]:
print(ws['A1'].value)

Código do Inventário para vinculo com apontamento


Carregando os valores no script

In [17]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
#     if obj['Rodovia'] is not None:           
   
#         obj['img'] = [a for a in ws._images if a.anchor._from.row == index]

#         try:
#             for a in obj['img']:
#                 a.name = 'CSG'
# #             obj['img'][0].name='Verso'
#         except:
#             pass

        
    values.append(obj)

Teste de valores importados da planilha

In [18]:
ficha_path = r'csg_monitoracao_2024/'

In [19]:
for a in values:
    a['img'] = []
#         print(a)
    for i in range(1,11):
        # Imagem Geral
        if a.get('Foto_'+str(i)) is not None:

            a['img'].append({
                'foto': ficha_path+a.get('Foto_'+str(i)),
                'data_foto': a.get('Data Foto_'+str(i)),
                'tipo_foto': a.get('Tipo Foto_'+str(i)),
                'des_foto': a.get('Descrição Foto_'+str(i))
            })

In [20]:
len(values),values[1]

(998,
 {'Código do Inventário para vinculo com apontamento': None,
  'Latitude': -29.664653479121625,
  'Longitude': -51.26029604114592,
  'km': '000+370',
  'km final': None,
  'km de Projeto': None,
  'km final de Projeto': None,
  'Status': 'Executado',
  'Equipe': 'Egati',
  'Encontrado em': None,
  'Executado em': None,
  'Prazo': None,
  'Sentido': 'Crescente',
  'Classe': 'Sinalização Vertical',
  'Faixa': 'Não se aplica',
  'Rodovia': 'ERS-122',
  'Código': 'INDICATIVA',
  'Lado': 'Esquerdo',
  'Data da medição atual': datetime.datetime(2024, 1, 4, 0, 0),
  'Fabricante': 'SINASC',
  'Data de fabricação': datetime.datetime(2023, 10, 1, 0, 0),
  'Tipo de suporte': 'PERFIL C METÁLICO',
  'Estado do suporte': 'BOM',
  'Observações': None,
  'Monitoração': '1 MONITORAÇÃO',
  'Recurso_1': None,
  'Quantidade_1': None,
  'Recurso_2': None,
  'Quantidade_2': None,
  'Recurso_3': None,
  'Quantidade_3': None,
  'Recurso_4': None,
  'Quantidade_4': None,
  'Recurso_5': None,
  'Quantidad

Definindo valores da importação

In [21]:
# company = Company.objects.get(name='Caminhos da Serra Gaúcha')
company = Company.objects.get(name='Caminhos da Serra Gaúcha')
firm=Firm.objects.get(uuid='7e50d5c5-4019-4165-ab39-39782fec0118')
#Classe Sinalização Vertical
occurrence_type=OccurrenceType.objects.get(uuid='eb083398-37d5-43a0-b245-423d3c847afe')

user=User.objects.get(username='rlcs')
status=ServiceOrderActionStatus.objects.get(companies=company, name='Executado')
# step=ApprovalStep.objects.get(approval_flow__company=company, name='Em Elaboração')
# firm=Firm.objects.get(uuid='0edd2ecc-8d5e-4444-898b-0069a3e9c354')

sign_name='Retrorrefletância'
company,occurrence_type,firm

(<Company: 4edb7778-e350-4e77-8e1e-de5f87b1da7f: Caminhos da Serra Gaúcha>,
 <OccurrenceType: Sinalização Vertical - ['Demo Concessionárias', 'Caminhos da Serra Gaúcha']>,
 <Firm: [4edb7778-e350-4e77-8e1e-de5f87b1da7f: Caminhos da Serra Gaúcha] 7e50d5c5-4019-4165-ab39-39782fec0118: Egati>)

In [22]:
# for a in tqdm(values):
#     a['Retrorrefletância'] = list(filter(None, [{'color': get_sign(sign_name,'Cor',occurrence_type,a['Cor_1'].capitalize()) if a['Cor_1'] is not None else None, 'first': a['Leitura 1 (cd/lx/m2)_1'], 'second': a['Leitura 2 (cd/lx/m2)_1'], 'third': a['Leitura 3 (cd/lx/m2)_1'], 'fourth': a['Leitura 4 (cd/lx/m2)_1'], 'fifth': a['Leitura 5 (cd/lx/m2)_1'], 'average': a['Média das leituras_1'], 'initial_value': a['Valor Inicial_1'], 'residual_value': a['Valor Residual_1']} if a['Cor_1'] is not None else None,
#                                   {'color': get_sign(sign_name,'Cor',occurrence_type,a['Cor_2'].capitalize()) if a['Cor_2'] is not None else None, 'first': a['Leitura 1 (cd/lx/m2)_2'], 'second': a['Leitura 2 (cd/lx/m2)_2'], 'third': a['Leitura 3 (cd/lx/m2)_2'], 'fourth': a['Leitura 4 (cd/lx/m2)_2'], 'fifth': a['Leitura 5 (cd/lx/m2)_2'], 'average': a['Média das leituras_2'], 'initial_value': a['Valor Inicial_2'], 'residual_value': a['Valor Residual_2']} if a['Cor_2'] is not None else None,
#                                   {'color': get_sign(sign_name,'Cor',occurrence_type,a['Cor_3'].capitalize()) if a['Cor_3'] is not None else None, 'first': a['Leitura 1 (cd/lx/m2)_3'], 'second': a['Leitura 2 (cd/lx/m2)_3'], 'third': a['Leitura 3 (cd/lx/m2)_3'], 'fourth': a['Leitura 4 (cd/lx/m2)_3'], 'fifth': a['Leitura 5 (cd/lx/m2)_3'], 'average': a['Média das leituras_3'], 'initial_value': a['Valor Inicial_3'], 'residual_value': a['Valor Residual_3']} if a['Cor_3'] is not None else None]))

In [46]:
for a in occurrence_type.form_fields['fields']:
    print(a['apiName'])
    if a['dataType'] == 'arrayOfObjects':
        for inner in a['innerFields']:
            print('\t',inner['apiName'],inner['displayName'])

code
side
datemeasurement
manufacturer
datemanufacture
typesupport
statesupport
retro
	 color Cor
	 typeOfFilm Tipo de Película
	 first Leitura 1 (cd/lx/m²)
	 second Leitura 2 (cd/lx/m²)
	 third Leitura 3 (cd/lx/m²)
	 fourth Leitura 4 (cd/lx/m²)
	 fifth Leitura 5 (cd/lx/m²)
	 average Média das leituras
	 initialValue Valor Inicial
	 residualValue Valor Residual
notes
monitoringId


In [47]:
# verifica se cada uma das chaves 'Cor_1', 'Cor_2' e 'Cor_3' está presente e não é None antes de tentar acessar

for a in tqdm(values):
    retrorrefletancia_list = []
    
    if 'Retrorrefletância_1: Cor' in a.keys() and a['Retrorrefletância_1: Cor'] is not None:
        retrorrefletancia_list.append({'color': get_sign(sign_name, 'Cor', occurrence_type, a['Retrorrefletância_1: Cor'].capitalize()),
                                       'first': a['Retrorrefletância_1: Leitura 1 (cd/lx/m²)'],
                                       'second': a['Retrorrefletância_1: Leitura 2 (cd/lx/m²)'],
                                       'third': a['Retrorrefletância_1: Leitura 3 (cd/lx/m²)'],
                                       'fourth': a['Retrorrefletância_1: Leitura 4 (cd/lx/m²)'],
                                       'fifth': a['Retrorrefletância_1: Leitura 5 (cd/lx/m²)'],
                                       'average': a['Retrorrefletância_1: Média das leituras'],
                                       'initial_value': a['Retrorrefletância_1: Valor Inicial'],
                                       'residual_value': a['Retrorrefletância_1: Valor Residual'],
                                      'type_of_film': get_sign(sign_name, 'Tipo de Película', occurrence_type, a['Retrorrefletância_1: Tipo de Película'])})
    
#     if 'Retrorrefletância_2: Cor' in a.keys() and a['Retrorrefletância_2: Cor'] is not None:
#         retrorrefletancia_list.append({'color': get_sign(sign_name, 'Cor', occurrence_type, a['Cor_2'].capitalize()),
#                                        'first': a['Leitura 1 (cd/lx/m2)_2'],
#                                        'second': a['Leitura 2 (cd/lx/m2)_2'],
#                                        'third': a['Leitura 3 (cd/lx/m2)_2'],
#                                        'fourth': a['Leitura 4 (cd/lx/m2)_2'],
#                                        'fifth': a['Leitura 5 (cd/lx/m2)_2'],
#                                        'average': a['Média das leituras_2'],
#                                        'initial_value': a['Valor Inicial_2'],
#                                        'residual_value': a['Valor Residual_2']})
    
#     if 'Retrorrefletância_3: Cor' in a.keys() and a['Retrorrefletância_3: Cor'] is not None:
#         retrorrefletancia_list.append({'color': get_sign(sign_name, 'Cor', occurrence_type, a['Cor_3'].capitalize()),
#                                        'first': a['Leitura 1 (cd/lx/m2)_3'],
#                                        'second': a['Leitura 2 (cd/lx/m2)_3'],
#                                        'third': a['Leitura 3 (cd/lx/m2)_3'],
#                                        'fourth': a['Leitura 4 (cd/lx/m2)_3'],
#                                        'fifth': a['Leitura 5 (cd/lx/m2)_3'],
#                                        'average': a['Média das leituras_3'],
#                                        'initial_value': a['Valor Inicial_3'],
#                                        'residual_value': a['Valor Residual_3']})
    
    a['Retrorrefletância'] = list(filter(None, retrorrefletancia_list))


  0%|          | 0/998 [00:00<?, ?it/s]

Criando itens de Inventário

In [48]:
values[0]

{'Código do Inventário para vinculo com apontamento': None,
 'Latitude': -29.66598406434059,
 'Longitude': -51.257022907957435,
 'km': '000+000',
 'km final': None,
 'km de Projeto': None,
 'km final de Projeto': None,
 'Status': 'Executado',
 'Equipe': 'Egati',
 'Encontrado em': None,
 'Executado em': None,
 'Prazo': None,
 'Sentido': 'Crescente',
 'Classe': 'Sinalização Vertical',
 'Faixa': 'Não se aplica',
 'Rodovia': 'ERS-122',
 'Código': 'M.K.',
 'Lado': 'Esquerdo',
 'Data da medição atual': datetime.datetime(2024, 1, 4, 0, 0),
 'Fabricante': 'FS METAL',
 'Data de fabricação': None,
 'Tipo de suporte': 'POSTE MADEIRA RETÂNGULAR',
 'Estado do suporte': 'BOM',
 'Observações': None,
 'Monitoração': '1 MONITORAÇÃO',
 'Recurso_1': None,
 'Quantidade_1': None,
 'Recurso_2': None,
 'Quantidade_2': None,
 'Recurso_3': None,
 'Quantidade_3': None,
 'Recurso_4': None,
 'Quantidade_4': None,
 'Recurso_5': None,
 'Quantidade_5': None,
 'Recurso_6': None,
 'Quantidade_6': None,
 'Recurso_7': N

In [26]:
objects=[]
linha=[]

for index, a in enumerate(tqdm(values)):
    km = float(a['km'].replace("+", "."))
    
#     long=str(a['Longitude_Inicial'])[:3]+'.'+str(a['Longitude_Inicial'])[3:]
#     lat=str(a['Latitude_Inicial'])[:3]+'.'+str(a['Latitude_Inicial'])[3:]
#     pista = 'Pista Dupla' if a['Pista'].strip().lower() == 'dupla' or a['Pista'] == 'Dupla' else 'Pista Simples'
#     road,_,point,km =point_to_km(float(long),float(lat),road_name)
    point, road = get_road_coordinates(a['Rodovia'].strip(), km, str(get_connected_reference(company, 'reporting', 'direction', a['Sentido'].capitalize())), company)
    if road_name not in road.name:
        print(f"Rodovia {road} - km:{km} - Linha: {index+2}")
    try:
        
        objects.append((Reporting(
                    company=company,
                    occurrence_type=occurrence_type,
#                     lane=str(get_connected_reference(company, 'reporting', 'lane', pista)),
                    lane=str(get_connected_reference(company, 'reporting', 'lane', a['Faixa'])),
                    road=road,
                    road_name=road.name,
                    direction=str(get_connected_reference(company, 'reporting', 'direction', a['Sentido'])),
#                     direction=str(get_connected_reference(company, 'reporting', 'direction', 'Não se aplica')),
                    created_by=user,
                    found_at=datetime.datetime(2024, 3, 20, 0, 0),
                    km=float(km),
#                     end_km=float(end_km) if a['Longitude_Final'] is not None and a['Latitude_Final'] else None,
                    point=point,
                    firm=firm,
                    status=status,
                    form_data={
                        'code': a['Código'] if a['Código'] is not None else '',
#                         'type_of_film': get_value('Tipo de Película',occurrence_type,a['Tipo de Película']) if a['Tipo de Película'] is not None else None,
                        'side': get_value('Lado',occurrence_type,str(a['Lado']).capitalize()) if a['Lado'] is not None else None,
                        'datemeasurement': a['Data da medição atual'].strftime('%Y-%m-%dT03:00:00.000Z') if a['Data da medição atual'] is not None and a['Data da medição atual'] != '' and a['Data da medição atual'] != 'A-52' else None,
                        'manufacturer': a['Fabricante'] if a['Fabricante'] is not None else None,
                        'datemanufacture': a["Data de fabricação"].strftime('%Y-%m-%dT03:00:00.000Z') if a['Data de fabricação'] is not None and a['Data de fabricação'] != '' else None,
#                         'datemanufacture': a['Data de fabricação'].strftime('%Y-%m-%dT03:00:00.000Z') if a['Data de fabricação'] is not None and a['Data de fabricação'] != '' else None,
                        'typesupport': a['Tipo de suporte'] if a['Tipo de suporte'] is not None else None,
                        'statesupport': a['Estado do suporte'] if a['Estado do suporte'] is not None else None,
                        'retro': a['Retrorrefletância'] if a['Retrorrefletância'] is not None else None,
                        'notes': a['Observações'] if a['Observações'] is not None else ''
                        
                    },
                    ),a['img']))
    except Exception as e:
        linha.append(index+2)
        print(f'Arquivo"{filename}" - Rodovia {road} - Km:{km} - Linha: {index+2} - {objects.eu(e)}')

  0%|          | 0/998 [00:00<?, ?it/s]

In [27]:
len(objects),objects[0][0].__dict__,objects[0][1]

(998,
 {'_state': <django.db.models.base.ModelState at 0x7059c89fee50>,
  'uuid': UUID('e9b446dd-d27d-4de7-bc1f-8cf8d3f41e38'),
  'number': '',
  'company_id': UUID('4edb7778-e350-4e77-8e1e-de5f87b1da7f'),
  'road_name': 'ERS-122',
  'road_id': 2059,
  'km': 0.0,
  'end_km': None,
  'km_reference': None,
  'project_km': 0,
  'project_end_km': None,
  'point': <Point object at 0x7059c92b7f10>,
  'direction': '0',
  'lane': '99',
  'track': None,
  'branch': None,
  'address': {},
  'created_by_id': UUID('508a66f3-3f58-4c4e-bc9a-fbba6078c02d'),
  'firm_id': UUID('7e50d5c5-4019-4165-ab39-39782fec0118'),
  'occurrence_type_id': UUID('eb083398-37d5-43a0-b245-423d3c847afe'),
  'form_data': {'code': 'M.K.',
   'side': '2',
   'datemeasurement': '2024-01-04T03:00:00.000Z',
   'manufacturer': 'FS METAL',
   'datemanufacture': None,
   'typesupport': 'POSTE MADEIRA RETÂNGULAR',
   'statesupport': 'BOM',
   'retro': [],
   'notes': ''},
  'form_metadata': {},
  'executed_at': None,
  'created_at'

# Script seguinte ira salvar os dados no bd

In [28]:
from django.db import transaction
import uuid

In [29]:
fotos_apt=[]
apt=[]
for reporting, images in tqdm(objects):
    apt.append(reporting)
    for image in images:
        reporting_file = ReportingFile(
            created_by=user,
            reporting=reporting,
            description=image.get('des_foto') if image.get('des_foto') else 'CSG',
            datetime=image.get('data_foto', datetime.datetime(2000, 1, 1, 0, 0)) if isinstance(image.get('data_foto'), datetime.datetime) else datetime.datetime(2000, 1, 1, 0, 0),
            kind=image.get('tipo_foto', 'outro').lower() if image.get('tipo_foto') is not None else 'outro',
            km=reporting.km,
            is_shared=False,
            point=reporting.point,
            upload=image.get('foto')
        )
        try:
            reporting_file.upload.size
            fotos_apt.append(reporting_file)
#             reporting_file.save()
        except Exception as e:
            print(reporting, reporting_file, e)
# print(reporting.number)

  0%|          | 0/998 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [75]:
len(apt),len(fotos_apt),fotos_apt[0].__dict__

(998,
 3556,
 {'_state': <django.db.models.base.ModelState at 0x7dd5b49b9370>,
  'uuid': UUID('c8f6496e-1155-4a2d-a12d-30e604fc6e87'),
  'reporting_id': UUID('5b50913c-de5e-4e52-b94e-96bd6fb0cff6'),
  'description': None,
  'md5': '',
  'upload': <FieldFile: csg_monitoracao_2024/Vertical_000+000_409_CaminhosDaSerraGaucha_20231104_073011.jpg>,
  'uploaded_at': datetime.datetime(2024, 3, 21, 13, 13, 10, 309819, tzinfo=<UTC>),
  'datetime': datetime.datetime(2000, 1, 1, 0, 0),
  'created_by_id': UUID('508a66f3-3f58-4c4e-bc9a-fbba6078c02d'),
  'include_dnit': True,
  'include_rdo': False,
  'km': 0.0,
  'point': <Point object at 0x7dd5bd1dd510>,
  'kind': 'outro',
  'is_shared': False})

In [74]:
fot = bulk_create_with_history(fotos_apt, ReportingFile, default_user=user)
apontamentos = bulk_update_with_history(apt,Reporting, use_django_bulk=True,batch_size=100,user=user)

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField ReportingFile.datetime received a naive datetime (2000-01-01 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


IntegrityError: null value in column "description" violates not-null constraint
DETAIL:  Failing row contains (c8f6496e-1155-4a2d-a12d-30e604fc6e87, null, csg_monitoracao_2024/Vertical_000+000_409_CaminhosDaSerraGaucha_..., 2024-03-21 13:13:10.309819+00, 508a66f3-3f58-4c4e-bc9a-fbba6078c02d, 5b50913c-de5e-4e52-b94e-96bd6fb0cff6, t, 0, 0101000020E610000011108EC3DEA049C07B7D3EB28AAA3DC0, f, 2000-01-01 02:00:00+00, , outro, f).


In [73]:
ficha_path

'csg_monitoracao_2024/'

Anexando as imagens

In [26]:
for reporting, images in tqdm(objects):
    for image in images:
        reporting_file = ReportingFile(
            created_by=user,
            reporting=reporting,
            description=image.name,
            km=reporting.km,
            point=reporting.point
        )
        reporting_file.save()
        image_io = BytesIO(image._data())
        im = Image.open(image_io)
        if im.mode in ("RGBA", "P"):
            im = im.convert("RGB")
        thumb_io = BytesIO()
        im.save(thumb_io, format='jpeg', quality=90)
        reporting_file.upload.save(image.name + '.jpeg', thumb_io)

  0%|          | 0/127 [00:00<?, ?it/s]

In [69]:
Reporting.objects.filter(company=company,occurrence_type=occurrence_type,road__name__contains='122').delete()

(10105, {'reportings.ReportingFile': 5061, 'reportings.Reporting': 5044})

In [6]:
for a in reps:
    print(a.number)

CSG-Inv-2023.02059
CSG-Inv-2023.02060
CSG-Inv-2023.02506
CSG-Inv-2023.02535


AttributeError: 'DailyReportContractUsage' object has no attribute 'count'

(1, {'daily_reports.DailyReportContractUsage': 1})

In [56]:
company = Company.objects.get(name="Caminhos da Serra Gaúcha")

In [57]:
filename = "retificacaoPistaDupla122.xlsx"
wb = load_workbook(filename)
ws = wb[wb.sheetnames[1]]
ws

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


<Worksheet "Sinalização Vertical">

In [58]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    values.append(obj)
values[0]

{'km': None,
 'km final': None,
 'km de Projeto': None,
 'km final de Projeto': None,
 'Latitude_Inicial': -29.6650463888889,
 'Longitude_Inicial': -51.258245,
 'Latitude_Final': None,
 'Longitude_Final': None,
 'Status': 'Identificado',
 'Equipe/Empreiteira': 'Cadastro incial',
 'Encontrado em': None,
 'Executado em': None,
 'Sentido': 'Crescente',
 'Pista': 'Dupla',
 'Classe': 'Sinalização Vertical',
 'Lado': 'Direito',
 'Faixa': 'Não se aplica',
 'Rodovia': 'ERS-122',
 'Código': 'DEL',
 'Data da medição atual': datetime.datetime(2023, 1, 19, 0, 0),
 'Fabricante': 'SINARODO',
 'Data de fabricação': None,
 'Tipo de suporte': 'MADEIRA',
 'Estado do suporte': 'DANIFICADO',
 'Tipo de Película': 'I',
 'Cor_1': 'AMARELO',
 'Leitura 1 (cd/lx/m2)_1': 6.5,
 'Leitura 2 (cd/lx/m2)_1': 4.5,
 'Leitura 3 (cd/lx/m2)_1': 5.7,
 'Leitura 4 (cd/lx/m2)_1': 5.4,
 'Leitura 5 (cd/lx/m2)_1': 5,
 'Média das leituras_1': 5.42,
 'Valor mínimo_1': 50,
 'Valor Residual_1': 25,
 'Cor_2': None,
 'Leitura 1 (cd/lx/

In [59]:
road=Road.objects.get(id=2059)
road

<Road: 2059: ERS-122>

In [60]:
point_to_km(-51.0483375,-28.6201603,'122')

(<Road: 2104: ERS-122>, 'ERS-122', <Point object at 0x7fd091592370>, 167.745)

In [55]:
values[0]['Latitude_Inicial'],values[0]['Longitude_Inicial']
road,_,point,km =point_to_km(float(values[0]['Longitude_Inicial']),float(values[0]['Latitude_Inicial']),road)
road,point,km

(<Road: 2059: ERS-122>, <Point object at 0x7ffab6f0c400>, 0.195)

In [62]:
rep_edit=[]
rep_edit_filter=[]
for a in tqdm(values):
    road,_,point,km =point_to_km(float(a['Longitude_Inicial']),float(a['Latitude_Inicial']),road)
    direction = str(get_connected_reference(company, 'reporting', 'direction', a['Sentido']))
    lane='Pista Dupla' if a['Pista'].strip().lower() == 'dupla' or a['Pista'] == 'Dupla' else 'Pista Simples'
    
    try:
        rep = Reporting.objects.get(point=point,company=company,occurrence_type=occurrence_type)
    except:
        reps=Reporting.objects.filter(point=point,company=company,occurrence_type=occurrence_type)
        for b in reps:
            rep_edit_filter.append(b)
        
        
#     if '122' not in road.name:
#         rep.lane = str(get_connected_reference(company, 'reporting', 'lane', lane))
#         rep_edit.append(rep)
#         print(road,rep,km,a['Sentido'])
#     else:
#         try:
#             rep.lane = str(get_connected_reference(company, 'reporting', 'lane', lane))
#             rep.road=road
#             rep.road_name=road.name
#             rep_edit.append(rep)
#         except:
#             print(road,rep,km,direction,lane)

  0%|          | 0/5044 [00:00<?, ?it/s]

In [67]:
len(set(rep_edit_filter))

2217

In [64]:
for c in rep_edit_filter:
    print(c)

[Caminhos da Serra Gaúcha] CSG-Inv-2023.06831 - 2023-06-23 15:20:53.308639+00:00
[Caminhos da Serra Gaúcha] CSG-Inv-2023.06832 - 2023-06-23 15:20:54.770330+00:00
[Caminhos da Serra Gaúcha] CSG-Inv-2023.06831 - 2023-06-23 15:20:53.308639+00:00
[Caminhos da Serra Gaúcha] CSG-Inv-2023.06832 - 2023-06-23 15:20:54.770330+00:00
[Caminhos da Serra Gaúcha] CSG-Inv-2023.06867 - 2023-06-23 15:21:46.392382+00:00
[Caminhos da Serra Gaúcha] CSG-Inv-2023.06869 - 2023-06-23 15:21:49.466915+00:00
[Caminhos da Serra Gaúcha] CSG-Inv-2023.06868 - 2023-06-23 15:21:47.837510+00:00
[Caminhos da Serra Gaúcha] CSG-Inv-2023.06867 - 2023-06-23 15:21:46.392382+00:00
[Caminhos da Serra Gaúcha] CSG-Inv-2023.06869 - 2023-06-23 15:21:49.466915+00:00
[Caminhos da Serra Gaúcha] CSG-Inv-2023.06868 - 2023-06-23 15:21:47.837510+00:00
[Caminhos da Serra Gaúcha] CSG-Inv-2023.06867 - 2023-06-23 15:21:46.392382+00:00
[Caminhos da Serra Gaúcha] CSG-Inv-2023.06869 - 2023-06-23 15:21:49.466915+00:00
[Caminhos da Serra Gaúcha] C

In [30]:
user=User.objects.get(username="rlcs")

In [31]:
apt=bulk_update_with_history(rep_edit,Reporting, use_django_bulk=True,batch_size=1250,user=user)

In [53]:
rep=Reporting.objects.filter(company=company,occurrence_type=occurrence_type, road__name__contains='122').filter(form_data__code='DEL').filter(form_data__typesupport='MADEIRA')

In [54]:
rep.count()

TypeError: Object of type datetime is not JSON serializable

In [47]:
rep[3].form_data

IndexError: list index out of range